In [35]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import pandas as pd
from pathlib import Path
import pickle
# import logging
# log = logging.getLogger()
# log.setLevel(logging.DEBUG)

from io_utils import *
from trials_table import * 
%load_ext autoreload
%autoreload 2
dj.blob.use_32bit_dims = True # necessary for pd.blob read

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
#### MULTI SESSION

animal_id = 'R502'
bdata = dj.create_virtual_module("bdata", "bdata")
subject_session_key = {"ratname": animal_id}

pd_blobs0 = (bdata.Sessions & subject_session_key).fetch(
    "protocol_data", as_dict=True
)

sess_ids0, dates0, trials0 = (bdata.Sessions & subject_session_key).fetch(
    "sessid", "sessiondate", "n_done_trials"
)

peh_blobs0 = (bdata.ParsedEvents * bdata.Sessions & subject_session_key).fetch('peh', as_dict=True)



In [49]:
dict_for_saving = {'pds':pd_blobs0, 'sess_ids': sess_ids0, 'dates' : dates0, 'trials': trials0, 'pehs' : peh_blobs0}

output = open('data/pd_peh.pkl', 'wb')
pickle.dump(dict_for_saving, output)
output.close()

In [ ]:

with open('data/pd_peh.pkl', 'rb') as fh:
    pd_peh_dict = pickle.load(fh)


In [50]:
pd_blobs0 = pd_peh_dict['pds']
peh_blobs0 = pd_peh_dict['pehs']
trials0 = pd_peh_dict['trials']
sess_ids0 = pd_peh_dict['sess_ids']
dates0 = pd_peh_dict['dates']

In [51]:

trial_filter = (trials0 != 0) & (trials0 != 1)
pd_blobs1, peh_blobs1, trials1, sess_ids1, dates1 = drop_sessions(
    pd_blobs0, peh_blobs0, trials0, sess_ids0, dates0, trial_filter
)


pd_dicts0 = convert_to_dict(pd_blobs1)
reformat_pd_dict(pd_dicts0)

peh_dicts0 = convert_to_dict(peh_blobs1)

assert len(pd_dicts0) == len(peh_dicts0) == len(trials1) == len(sess_ids1) == len(dates1) , (
    "number of sessions does not match, assumptions of code below are broken!"
)

pd_dicts, peh_dicts, trials, sess_ids, dates = find_and_fix_len_errors(
    pd_dicts0, peh_dicts0, trials1, sess_ids1, dates1
)


peh_dicts_for_df = get_peh_vars_for_df(peh_dicts, trials, sess_ids)

df = generate_trials_df(animal_id, pd_dicts, peh_dicts_for_df, trials, sess_ids, dates)



dropping 1 sessions of 125
dropping 0 sessions of 124


In [52]:
df

,trial,animal_id,date,sess_id,hits,violations,temperror,sides,result,sa,...,stage,is_match,sound_pair,early_spoke,t_start,t_end,cpoke_in,cpoke_out,spoke_in,go_time
0,1,R502,2022-07-19,879475,0,0,1,r,4,3.0,...,1,False,"3.0, 12.0",False,398.610404,908.022504,896.475104,896.705304,901.980104,897.575104
1,2,R502,2022-07-19,879475,1,0,0,r,1,3.0,...,1,False,"3.0, 12.0",False,908.465804,1547.456404,1533.383904,1533.627104,1541.414004,1533.883904
2,3,R502,2022-07-19,879475,0,0,1,r,4,3.0,...,1,False,"3.0, 12.0",False,1547.456567,2369.620867,2360.900767,2361.042967,2363.578467,2361.400767
3,4,R502,2022-07-19,879475,1,0,0,l,1,3.0,...,1,True,"3.0, 3.0",False,2369.620967,2524.491867,2517.808567,2517.876467,2518.450267,2518.308567
4,5,R502,2022-07-19,879475,1,0,0,r,1,3.0,...,1,False,"3.0, 12.0",False,2524.491967,2564.889467,2550.768067,2550.899167,2558.847067,2551.268067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34312,315,R502,2022-11-23,897087,1,0,0,r,1,12.0,...,8,False,"12.0, 3.0",True,3921.262903,3924.606003,3921.263103,3921.434903,3922.073203,3922.073103
34313,316,R502,2022-11-23,897087,1,0,0,r,1,12.0,...,8,False,"12.0, 3.0",True,3930.259203,3933.425703,3930.259403,3930.431303,3931.069503,3931.069403
34314,317,R502,2022-11-23,897087,1,0,0,r,1,12.0,...,8,False,"12.0, 3.0",True,3938.552303,3941.764603,3938.627203,3938.800203,3939.437303,3939.437203
34315,318,R502,2022-11-23,897087,1,0,0,l,1,3.0,...,8,True,"3.0, 3.0",True,3946.905903,3950.135703,3946.920803,3947.105303,3947.730903,3947.730803


In [3]:

# #### SINGLE SESSION ####
# session_key = {'sessid': 884257}

# prb = (bdata.Sessions & session_key).fetch(
#     "protocol_data", as_dict=True
# )

# s_id, d, t = (bdata.Sessions & session_key).fetch(
#     "sessid", "sessiondate", "n_done_trials"
# )
# prd = convert_to_dict(prb)
# pd_prepare_dict_for_df(prd)
# session_key = {'sessid': s_id[0]}
# pehb = (bdata.ParsedEvents & session_key).fetch("peh", as_dict=True)
# pehd = bt.transform_blob(pehb[0]['peh'])

